In [3]:
import requests
import pandas as pd

In [5]:
gr_metadata_merged = pd.read_csv("../data/gr_metadata_merged.csv")
gr_metadata_merged.head(5)

,doc_id,author,title,not_before,not_after,LAGT4-1?,GLAUX?,OGA?,EXPRECCE?,lagt4-1_author,...,exprecce_author,exprecce_title,exprecce_not_after,exprecce_not_before,exprecce_edition,exprecce_edition_link,oga_id,oga_print_edition,glaux_tlg_ids,glaux_tokens_sum
0,ggm0001.ggm001,Anonymous,Anametresis Pontou,1.0,400.0,False,False,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1607.0,"Anonymous. Geographi graeci minores, Volume 1....",[nan],0.0
1,ogl0001.ogl001,Pinytus,De Epistola Pinyti ad Dionysium,101.0,200.0,True,False,True,False,Pinytus,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"Pinytus, Saint, Bishop of Knossos. Reliquiae S...",[nan],0.0
2,pta0001.pta001,Severian of Gabala,De fide et lege naturae,400.0,409.0,False,False,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1694.0,"Vatikan, Biblioteca Apostolica Vaticana, graec...",[nan],0.0
3,pta0001.pta002,Severian of Gabala,De paenitentia et compunctione,400.0,409.0,False,False,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1695.0,"Jean-Paul Migne (ed.), Patrologia Graeca. Volu...",[nan],0.0
4,pta0001.pta003,Severian of Gabala,In ascensionem domini nostri Iesu Christi et i...,400.0,409.0,False,False,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1696.0,"Richard W. Bishop/Nathalie Rambault, Severian ...",[nan],0.0


In [2]:
def wd_by_an_identifier(property, idstring):
    """
    Query Wikidata for an entity with the given TLG author ID (property P1266).
    Returns JSON results from the WDQS endpoint.
    """
    query = f"""
    SELECT ?item ?itemLabel WHERE {{
      ?item wdt:{property} "{idstring}" .
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """

    url = "https://query.wikidata.org/sparql"
    headers = {"Accept": "application/sparql-results+json"}

    response = requests.get(url, params={"query": query, "format": "json"}, headers=headers)
    response.raise_for_status()

    return response.json()

In [10]:
wd_by_an_identifier("P3576", "0086")

{'head': {'vars': ['item', 'itemLabel']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q868'},
    'itemLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Aristotle'}}]}}

In [6]:
doc_id = "tlg0086.tlg009"
doc_id[3:7]

'0086'

In [8]:
def get_wd_authors(doc_id):
    author_wd = ""
    if "tlg" in doc_id:
        tlg_author_id = doc_id[3:7]
        try:
            author_wd =  wd_by_an_identifier("P3576", tlg_author_id)["results"]["bindings"][0]["item"]["value"].rpartition("/")[2]
        except:
            pass
    return author_wd

In [10]:
gr_metadata_merged["author_wd"] = gr_metadata_merged["doc_id"].apply(get_wd_authors)

In [11]:
sum(gr_metadata_merged["author_wd"]!="")

491

In [13]:
gr_metadata_merged[gr_metadata_merged["author_wd"]!=""].sample(10)

,doc_id,author,title,not_before,not_after,LAGT4-1?,GLAUX?,OGA?,EXPRECCE?,lagt4-1_author,...,exprecce_title,exprecce_not_after,exprecce_not_before,exprecce_edition,exprecce_edition_link,oga_id,oga_print_edition,glaux_tlg_ids,glaux_tokens_sum,author_wd
769,tlg0062.tlg051,Lucian,Τυραννοκτόνος,140.0,149.0,True,True,True,False,Lucian,...,NaN,NaN,NaN,NaN,NaN,454.0,"Lucianus, Tyrannicida, Harmon, Harvard, 1962",['0062-051'],3396.0,Q177847
768,tlg0062.tlg050,Lucian,Θεῶν Ἐκκλησία,115.0,190.0,True,True,True,False,Lucian,...,NaN,NaN,NaN,NaN,NaN,1197.0,"Lucianus, Deorum concilium, Harmon, Harvard, 1962",['0062-050'],2145.0,Q177847
832,tlg0085.tlg002,Aeschylus,Πέρσαι,-471.0,-471.0,True,True,True,False,Aeschylus,...,NaN,NaN,NaN,NaN,NaN,507.0,"Aeschyli Tragoediae. Sidgwick, Arthur, editor....",['0085-002'],6256.0,Q40939
116,tlg0006.tlg009,Euripides,Ἡρακλῆς,-415.0,-415.0,True,True,True,False,Euripides,...,NaN,NaN,NaN,NaN,NaN,20.0,"Euripides. Euripidis Fabulae, Vol. II. Murray,...",['0006-009'],9632.0,Q48305
214,tlg0007.tlg084,Plutarchus,Aetia Romana et Graeca,1.0,200.0,True,True,False,False,Plutarchus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['0007-084a', '0007-084b']",22114.0,Q41523
374,tlg0014.tlg024,Demosthenes,κατὰ Τιμοκράτους,-352.0,-351.0,True,True,True,False,Demosthenes,...,NaN,NaN,NaN,NaN,NaN,237.0,"Demosthenis. Orationes. Vol. II, Part 1. Butch...",['0014-024'],17158.0,Q117253
1747,tlg2022.tlg009,Gregory of Nazianzus,De Filio (Orat. 29),329.0,390.0,True,False,True,False,Gregory of Nazianzus,...,NaN,NaN,NaN,NaN,NaN,947.0,"Gregorius Nazianzenus, The Five Theological Or...",[nan],0.0,Q44011
308,tlg0011.tlg002,Sophocles,Ἀντιγόνη,-442.0,-437.0,True,True,True,False,Sophocles,...,NaN,NaN,NaN,NaN,NaN,191.0,"Sophocles, Volume 1. Storr, Francis, editor. L...",['0011-002'],8866.0,Q7235
2014,tlg2966.tlg002,Phileas,Epistula ad Thmuitanos (ap. Eusebium),201.0,400.0,True,True,False,False,Phileas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['2966-002'],739.0,Q2348355
1204,tlg0540.tlg023,Lysias,κατὰ Παγκλέωνος ὅτι οὐκ ἦν Πλαταιεύς,-399.0,-398.0,True,True,True,False,Lysias,...,NaN,NaN,NaN,NaN,NaN,643.0,"Lysias. Lamb, W.R.M., editor. Cambridge, MA: H...",['0540-023'],889.0,Q310164


In [16]:
wds = gr_metadata_merged["author_wd"].unique().tolist()

In [17]:
len(wds)

152

In [18]:
headers = {
        "User-Agent": "MyResearchBot/0.1 (kase@ff.zcu.cz)"
    }
def get_entity_json(qid, headers = headers):
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()

In [19]:
def get_claim_value(qid, pid, headers=headers):
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbgetentities",
        "ids": qid,
        "props": "claims",
        "format": "json"
    }
    r = requests.get(url, params=params, headers=headers)
    r.raise_for_status()
    data = r.json()

    claims = data["entities"][qid]["claims"]

    if pid not in claims:
        return None  # property not present

    # take the first value
    mainsnak = claims[pid][0]["mainsnak"]
    datavalue = mainsnak.get("datavalue", {})
    return datavalue.get("value")

In [20]:
def get_author_viaf(wd):
    try:
        return get_claim_value(wd, "P214")
    except:
        return ""

In [21]:
wd_viaf = {}
for wd in wds:
    wd_viaf[wd] = get_author_viaf(wd)

In [22]:
wd_viaf

{'': '',
 'Q192638': '262974255',
 'Q336115': '265406304',
 'Q41683': '95161463',
 'Q59138': '41839141',
 'Q219484': '95161348',
 'Q48305': '265326651',
 'Q41523': '268955446',
 'Q294923': '100169636',
 'Q17892': '78770488',
 'Q221182': '36926979',
 'Q7235': '101760867',
 'Q6691': '224924963',
 'Q329342': '187475186',
 'Q117253': '268302412',
 'Q26825': '100225976',
 'Q365097': '265571975',
 'Q189597': '310526579',
 'Q43353': '20962036',
 'Q44233': '122220717',
 'Q363818': '22275493',
 'Q317089': '161234357',
 'Q335560': '26151776750618011460',
 'Q473421': '268224700',
 'Q314447': '264197546',
 'Q18813': '223078470',
 'Q129772': '89597697',
 'Q957548': '59900924',
 'Q463364': '250719105',
 'Q111302329': '15160007',
 'Q116960081': '67228',
 'Q8778': '44299175',
 'Q859': '108159964',
 'Q19558843': '148151776730118010511',
 'Q177847': '100218901',
 'Q2877686': '29151776728018010071',
 'Q87771481': '59886787',
 'Q1351420': '71234739',
 'Q31845': '100192636',
 'Q26204': '100218889',
 'Q4728

In [23]:
gr_metadata_merged["author_viaf"] = gr_metadata_merged["author_wd"].apply(lambda x: wd_viaf[x])

In [25]:
gr_metadata_merged.sample(10, random_state=0)

,doc_id,author,title,not_before,not_after,LAGT4-1?,GLAUX?,OGA?,EXPRECCE?,lagt4-1_author,...,exprecce_not_after,exprecce_not_before,exprecce_edition,exprecce_edition_link,oga_id,oga_print_edition,glaux_tlg_ids,glaux_tokens_sum,author_wd,author_viaf
175,tlg0007.tlg047,Plutarch,Alexander,96.0,120.0,True,True,True,False,Plutarch,...,NaN,NaN,NaN,NaN,77.0,"Plutarch. Plutarch's Lives, Vol. VII. Perrin, ...",['0007-047'],22724.0,,
264,tlg0007.tlg133,Plutarch,Πλατωνικὰ ζητήματα,96.0,120.0,True,True,True,False,Plutarch,...,NaN,NaN,NaN,NaN,153.0,"Plutarch. Plutarch's Morals, Vol. V. Goodwin, ...",['0007-133'],7146.0,,
2131,tlg5022.tlg005,Scholia in Euclidem,Scholia in Euclidis catoptrica (scholia vetera),301.0,400.0,False,False,True,False,NaN,...,NaN,NaN,NaN,NaN,1649.0,"Scholia in Euclidem, Scholia in Euclidis catop...",[nan],0.0,,
379,tlg0014.tlg029,Demosthenes,πρὸς Ἄφοβον ὑπὲρ Φάνου ψευδομαρτυριῶν γ΄,-361.0,-360.0,True,True,True,False,Demosthenes,...,NaN,NaN,NaN,NaN,242.0,"Demosthenis. Orationes. Vol. II, Part 2. Renni...",['0014-029'],4528.0,Q117253,268302412
317,tlg0012.tlg003,Homer,Ἐπιγράμματα,-799.0,-700.0,True,True,True,False,Homer,...,NaN,NaN,NaN,NaN,1682.0,"Homer. Hesiod, The Homeric Hymns and Homerica....",['0012-003'],26.0,Q6691,224924963
1407,tlg0636.tlg001,Socrates Epistulae,Epistulae,-468.0,-398.0,False,False,True,False,NaN,...,NaN,NaN,NaN,NaN,1361.0,Socrates Epistulae. Epistolographi Graeci. Her...,[nan],0.0,,
1192,tlg0540.tlg011,Lysias,"κατὰ θεομν́ηστου, Β",-383.0,-382.0,True,True,True,False,Lysias,...,NaN,NaN,NaN,NaN,631.0,"Lysias. Lamb, W.R.M., editor. Cambridge, MA: H...",['0540-011'],651.0,,
899,tlg0087.tlg011,Herodianus,Περὶ ὀρθογραφίας,101.0,200.0,True,False,True,False,Herodianus,...,NaN,NaN,NaN,NaN,538.0,"Aelius Herodianus, Περὶ ὀρθογραφίας, Grammatic...",[nan],0.0,,
182,tlg0007.tlg054,Plutarch,Demosthenes,96.0,116.0,True,True,True,False,Plutarch,...,NaN,NaN,NaN,NaN,84.0,"Plutarch. Plutarch's Lives, Vol. VII. Perrin, ...",['0007-054'],7787.0,,
107,tlg0005.tlg005,Theocritus,Epigrammata,-400.0,-201.0,True,True,False,False,Theocritus,...,NaN,NaN,NaN,NaN,NaN,NaN,['0005-005'],847.0,Q219484,95161348


In [27]:
gr_metadata_merged.to_csv("../data/gr_metadata_merged.csv", index=False)